# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Scrape events from a website
* Follow links to those events to retrieve further information
* Clean and store scraped data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [1]:
#Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [ ]:
#Open the inspect element feature in your browser

## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [2]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [7]:
response = requests.get("https://www.residentadvisor.net/events/us/colorado")
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
event_listings = soup.find('div', id="event-listing")
event_listings()

[<ul class="list" id="items">
 <li><p class="eventDate date"><a href="/events.aspx?ai=82&amp;v=day&amp;mn=10&amp;yr=2019&amp;dy=13"><span>Sun, 13 Oct 2019 /</span></a></p></li><li class=""><article class="event-item clearfix " itemscope="" itemtype="http://data-vocabulary.org/Event"><span style="display:none;"><time datetime="2019-10-13T00:00" itemprop="startDate">2019-10-13T00:00</time></span><a href="/events/1316688"><img height="76" src="/images/events/flyer/2019/10/us-1013-1316688-list.jpg" width="152"/></a><div class="bbox"><h1 class="event-title" itemprop="summary"><a href="/events/1316688" itemprop="url" title="Event details of Sorted Sunday: Ben UFO (Open to Close Set!)">Sorted Sunday: Ben UFO (Open to Close Set!)</a> <span>at <a href="/club.aspx?id=170917">Lodos Bar and Grill</a></span></h1><div class="grey event-lineup">Ben UFO</div><p class="attending"><span>17</span> Attending</p></div></article></li><li><p class="eventDate date"><a href="/events.aspx?ai=82&amp;v=day&amp;mn

In [9]:
entries = event_listings.findAll('li')
print(len(entries), entries[0])

19 <li><p class="eventDate date"><a href="/events.aspx?ai=82&amp;v=day&amp;mn=10&amp;yr=2019&amp;dy=13"><span>Sun, 13 Oct 2019 /</span></a></p></li>


In [10]:
#Successive exploration in function development
rows = []
for entry in entries:
    #Is it a date? If so, set current date.
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.columns = ['Event_Name', 'Venue', 'Event_Date', 'Number_of_Attendees']
df.head()


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Sorted Sunday: Ben UFO (Open to Close Set!),Lodos Bar and Grill,"Sun, 13 Oct 2019 /",17.0
1,Sorted Sunday: Ben UFO (Open to Close Set!),Lodos Bar and Grill,"Mon, 14 Oct 2019 /",17.0
2,Tennyson's Taco Tuesday feat. Tanya Leigh,Tennyson's Tap,"Tue, 15 Oct 2019 /",2.0
3,Patched Out,Fort Greene,"Thu, 17 Oct 2019 /",2.0
4,Red Rack'em,Neon Baby,"Thu, 17 Oct 2019 /",2.0


In [11]:
def scrape_events(events_page_url):
    #Your code here
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        #Is it a date? If so, set current date.
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.columns = ['Event_Name', 'Venue', 'Event_Date', 'Number_of_Attendees']
    return df
df

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Sorted Sunday: Ben UFO (Open to Close Set!),Lodos Bar and Grill,"Sun, 13 Oct 2019 /",17.0
1,Sorted Sunday: Ben UFO (Open to Close Set!),Lodos Bar and Grill,"Mon, 14 Oct 2019 /",17.0
2,Tennyson's Taco Tuesday feat. Tanya Leigh,Tennyson's Tap,"Tue, 15 Oct 2019 /",2.0
3,Patched Out,Fort Greene,"Thu, 17 Oct 2019 /",2.0
4,Red Rack'em,Neon Baby,"Thu, 17 Oct 2019 /",2.0
5,"Deep Club with Galcher Lustwerk, Wngdu, Ponyrok",Rhinoceropolis,"Fri, 18 Oct 2019 /",24.0
6,Phantoms,Bluebird Theater,"Fri, 18 Oct 2019 /",1.0
7,TroyBoi / G Jones,Red Rocks Amphitheatre,"Fri, 18 Oct 2019 /",1.0
8,"RECON DNB XVI Anniversary Feat. Digital, Coco ...",The Black Box,"Sat, 19 Oct 2019 /",2.0
9,RL Grime,Mission Ballroom,"Sat, 19 Oct 2019 /",2.0


## Write a Function to Retrieve the URL for the Next Page

In [12]:
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']

'/events/us/colorado/week/2019-10-20'

In [13]:
def next_page(url):
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    #Your code here
    return next_page_url

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [14]:
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/colorado"
while total_rows <= 1000:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df.head()

1000


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Sorted Sunday: Ben UFO (Open to Close Set!),Lodos Bar and Grill,"Sun, 13 Oct 2019 /",17.0
1,Sorted Sunday: Ben UFO (Open to Close Set!),Lodos Bar and Grill,"Mon, 14 Oct 2019 /",17.0
2,Tennyson's Taco Tuesday feat. Tanya Leigh,Tennyson's Tap,"Tue, 15 Oct 2019 /",2.0
3,Patched Out,Fort Greene,"Thu, 17 Oct 2019 /",2.0
4,Red Rack'em,Neon Baby,"Thu, 17 Oct 2019 /",2.0


## Summary 

Congratulations! In this lab, you successfully scraped a website for concert event information!